In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import os
import time

os.chdir(r"C:\Users\kimsoo\Desktop\source")

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### 웹 크롤링 함수

In [5]:
def generate_DB():
    df = pd.read_csv(r"C:\Users\kimsoo\Desktop\source\output.csv", encoding='utf-8')
    for page in range(1,4):
        print('현재 진행중인 페이지:',page)
        response = urlopen('https://edream.changwon.ac.kr/ko/program/all/list/all/'+ str(page))
        original_soup = BeautifulSoup(response, "html.parser")
        value = original_soup.find("ul", {"class": "columns-4"})
        html_text = str(value)
        items = original_soup.find_all('div', attrs={'data-role': 'item'})
        close = []
        for item in items:
            closed_label = item.find('label', class_='CLOSED')
            days_remaining = item.find('label', class_='APPROACH_CLOSING')
            schedule = item.find('label', class_='SCHEDULED')
            approaching = item.find('label', class_='APPROACHING')
            if closed_label:
                close.append('마감')
            elif days_remaining:
                D_day = days_remaining.find('b').text
                close.append(D_day)
            elif schedule:
                close.append('예정')
            elif approaching:
                close.append('임박')
            else:
                days_remaining = item.find('label', class_='OPEN')
                if days_remaining:
                    D_day = days_remaining.find('b').text
                    close.append(D_day)
                else:
                    close.append('')

        pattern = r'<a href="([^"]+)">'
        matches = re.findall(pattern, html_text)
        for url,close_ in zip(matches,close):
            url_response = urlopen("https://edream.changwon.ac.kr" + str(url))
            print('진행중인 사이트:',"https://edream.changwon.ac.kr" + str(url))
            url_soup = BeautifulSoup(url_response, "html.parser")
            html = str(url_soup.find("div", {"class": "title"}))

            soup = BeautifulSoup(html, 'html.parser')

            # 조회수 추출
            hits = soup.find('label', class_='hit').text.split()[0]

            # 유형 추출
            type_ = soup.find('label', class_='type').text

            # 프로그램명 추출
            program_name = soup.find('h4').text

            # 모집대상 추출
            target = soup.find('li', class_='target').find('span').text

            # 학년/성별 추출
            year_gender = soup.find_all('li')[2].find('span').text.strip()

            # 학과 추출
            major = soup.find('li', class_='department').find('span').text.strip()

            html = str(url_soup.find("div", {"class": "info"}))

            soup = BeautifulSoup(html, 'html.parser')

            # 부서 추출
            department = soup.find('div', class_='department').text.strip()

            # 카테고리 추출
            category = soup.find('div', class_='category').text.strip()

            # 이메일 추출
            email = soup.find('i', class_='fa fa-envelope').find_next_sibling('span').text

            # 연락처 추출
            phone = soup.find('i', class_='fa fa-phone').find_next_sibling('span').text

            # 위치 추출
            location = soup.find('i', class_='fa fa-map-marker').find_next_sibling('span').text

            #본문 추출
            description = url_soup.find("div", {"data-role": "wysiwyg-content"}).text.strip()

            #후기 추출
            review = url_soup.find("div", {"data-role": "review"}).text.strip()

            html = str(url_soup.find("div", {"class": "col-md-6"}))

            # JavaScript 부분에서 xAxis의 categories와 series의 data 값을 찾는 정규 표현식
            pattern_categories = r'xAxis:\{categories:\s*(\[.*?\])'
            pattern_data = r'series:\s*\[{"name":"핵심역량 지수","data":(\[.*?\])'

            # 정규 표현식을 사용하여 매치된 부분을 찾음
            match_categories = re.search(pattern_categories, html)
            match_data = re.search(pattern_data, html)

            if match_categories and match_data:
                # 매치된 문자열을 JSON으로 파싱
                categories = json.loads(match_categories.group(1))
                data = json.loads(match_data.group(1))

                # 최대 5개의 값을 각각의 변수에 저장
                data1, data2, data3, data4, data5 = data[:5]

            html = str(url_soup.find("div", {"class": "form"}))

            soup = BeautifulSoup(html, 'html.parser')
            if close_=='마감':
                for li in soup.select('li[data-open="0"]'):
                    # 제목 추출
                    subname = li.b.text.strip()

                    # 기간 추출
                    period = li.find_all('time')
                    start_time = period[0].text.strip()
                    end_time = period[1].text.strip()
                    period = f'{start_time} ~ {end_time}'

                    # 마일리지 추출
                    mileage = li.find('small').text.strip()

                    participants_info = li.find('span').text.strip()
                    current_participants_str, max_participants_str = participants_info.split(' / ')
                    current_participants = int(''.join(filter(str.isdigit, current_participants_str)))

                    # 최대인원이 '운영자승인'인 경우 처리
                    if '운영자승인' in max_participants_str:
                        max_participants = '운영자승인'
                    elif '무제한' in max_participants_str:
                        max_participants = '무제한'
                    else:
                        max_participants = int(''.join(filter(str.isdigit, max_participants_str)))

                    # 진행 상태 추출
                    status = li.find_all('p')[2].text.strip()
                    #추출된 자료들을 데이터프레임으로 만들기
                    data = pd.DataFrame({'프로그램명':program_name,'참가유형':type_,'조회수':hits,
                              '모집대상':target,'학년/성별':year_gender,'학과':major,'마감여부':close_,
                              '마일리지':mileage,'담당부서':department,'카테고리':category,
                              '이메일주소':email,'연락처':phone,'위치':location,'기간':period,
                              '신청인원':current_participants,'최대인원':max_participants,'진행 상태':status,
                              '부제목':subname,'지역형 리더역량':data1,'창의적 사고역량':data2,
                              '실용적 융복합역량':data3,'의사소통역량':data4,'글로벌역량':data5,
                              '본문':description,'후기':review,'사이트주소':"https://edream.changwon.ac.kr" + str(url)},index=[1])

                    #생성된 데이터 프레임과 기존의 데이터프레임 합치기
                    df = pd.concat([df, data], axis=0)
            else:
                for li in soup.select('li[data-open="1"]'):
                    # 제목 추출
                    subname = li.b.text.strip()

                    # 기간 추출
                    period = li.find_all('time')
                    start_time = period[0].text.strip()
                    end_time = period[1].text.strip()
                    period = f'{start_time} ~ {end_time}'

                    # 마일리지 추출
                    mileage = li.find('small').text.strip()

                    participants_info = li.find('span').text.strip()
                    current_participants_str, max_participants_str = participants_info.split(' / ')
                    current_participants = int(''.join(filter(str.isdigit, current_participants_str)))

                    # 최대인원이 '운영자승인'인 경우 처리
                    if '운영자승인' in max_participants_str:
                        max_participants = '운영자승인'
                    elif '무제한' in max_participants_str:
                        max_participants = '운영자승인'
                    else:
                        max_participants = int(''.join(filter(str.isdigit, max_participants_str)))

                    # 진행 상태 추출
                    status = li.find_all('p')[2].text.strip()

                    #추출된 자료들을 데이터프레임으로 만들기
                    data = pd.DataFrame({'프로그램명':program_name,'참가유형':type_,'조회수':hits,
                              '모집대상':target,'학년/성별':year_gender,'학과':major,'마감여부':close_,
                              '마일리지':mileage,'담당부서':department,'카테고리':category,
                              '이메일주소':email,'연락처':phone,'위치':location,'기간':period,
                              '신청인원':current_participants,'최대인원':max_participants,'진행 상태':status,
                              '부제목':subname,'지역형 리더역량':data1,'창의적 사고역량':data2,
                              '실용적 융복합역량':data3,'의사소통역량':data4,'글로벌역량':data5,
                              '본문':description,'후기':review,'사이트주소':"https://edream.changwon.ac.kr" + str(url)},index=[1])

                    #생성된 데이터 프레임과 기존의 데이터프레임 합치기
                    df = pd.concat([df, data], axis=0)

    print('크롤링 완료\n')

    df = df.drop(index=[0],axis=1).reset_index(drop=True)
    df = df[df['마감여부']!='마감']
    df_description = df.loc[:, ['프로그램명', '본문']]
    df = df.drop(['본문'],axis=1)
    df_T = df.T
    df.to_csv("Database.csv", encoding="utf-8-sig", index=False)
    df_T.to_csv("Database_T.csv", encoding="utf-8-sig", index=True)
    df_description.to_csv("Database_description.csv", encoding="utf-8-sig", index=False)

    print('데이터베이스 생성완료')

### 실시간 DB 업데이트

In [ ]:
while(1):
    wait_time = 3600
    generate_DB()
    time.sleep(wait_time)

현재 진행중인 페이지: 1
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10755
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10763
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10853
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10857
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10834
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10834
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10834
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10834
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10854
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10858
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10856
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10852
현재 진행중인 페이지: 2
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10810
진행중인 사이트: https://edream.changwon.ac.kr/ko/program/all/view/10823
진행중인 사이트: https://edream.changwon.ac.kr/ko/pro